In [18]:
# !pip install pymysql

In [19]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql, calendar, time, json
import requests
from datetime import datetime
from threading import Timer

In [20]:
class DBUpdater: 
    def __init__(self):        
        self.conn = pymysql.connect(host='localhost', user='root', \
            password='apple123!!', db='INVESTAR2', charset='utf8')
            
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info2 (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price2 (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        
        self.conn.commit()        
        self.codes = dict()
    
    def __del__(self):
        self.conn.close() 
        
    def read_krx_code(self):
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method='\
            'download&searchType=13'
            
        krx = pd.read_html(url, header=0)[0]  
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        
        return krx
    
    # 종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에(self.codes) 저장
    def update_comp_info(self):
        sql = "SELECT * FROM company_info2"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
            
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info2"
            curs.execute(sql)
            rs = curs.fetchone()
            print(rs, rs[0])
            
            today = datetime.today().strftime('%Y-%m-%d')
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]   
                    sql = f"REPLACE INTO company_info2 (code, company, last"\
                        f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)

                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info2 "\
                            f"VALUES ({code}, {company}, {today})")    
                
                self.conn.commit()    
    
    # 네이버 데이터 수집            
    def read_naver(self, code, company, pages_to_fetch):   
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
            html = BeautifulSoup(res.text, "lxml")
            
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1]             
            pages = min(int(lastpage), pages_to_fetch)
            df = pd.DataFrame()      
                  
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                tmpDf = pd.read_html(requests.get(pg_url, headers={'User-agent': 'Mozilla/5.0'}).text)[0]
                df = pd.concat([df, tmpDf], ignore_index=True)
                #df = pd.concat([df, pd.DataFrame([pd.read_html(requests.get(pg_url,
                #    headers={'User-agent': 'Mozilla/5.0'}).text)[0]])], ignore_index=True)
                
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                    format(tmnow, company, code, page, pages), end="\r")
                
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]           
        
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df   
    
    def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 저장"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price2 VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'\
                'price2 [OK]'.format(datetime.now().strftime('%Y-%m-%d'\
                ' %H:%M'), num+1, company, code, len(df)))
            
    def update_daily_price(self, pages_to_fetch):
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            
            self.replace_into_db(df, idx, code, self.codes[code])   
            return 
        
    def execute_daily(self):
        self.update_comp_info()
        
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100
                config = {'pages_to_fetch': pages_to_fetch}
                json.dump(config, out_file)
            
        self.update_daily_price(pages_to_fetch)
        
        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1, month=1, day=1,
                hour=17, minute=0, second=0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour=17,
                minute=0, second=0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour=17, minute=0,
                second=0)   
            
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        
        t = Timer(secs, self.execute_daily)
        t.start()

In [21]:
dbu = DBUpdater()
dbu.read_krx_code()
dbu.execute_daily()


/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_4785/976295043.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


(datetime.date(2024, 1, 11),) 2024-01-11
[2024-01-11 13:37] #0001 동화약품 (000020) : 1000 rows > REPLACE INTO daily_price2 [OK]


/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_4785/2856184959.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


(datetime.date(2024, 1, 11),) 2024-01-11


/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_4785/1648501702.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_4785/976295043.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


(datetime.date(2024, 1, 11),) 2024-01-11
(datetime.date(2024, 1, 11),) 2024-01-110100 pages are downloading...
[2024-01-11 17:41] #0001 동화약품 (000020) : 1000 rows > REPLACE INTO daily_price [OK]
[2024-01-11 17:41] #0001 동화약품 (000020) : 1000 rows > REPLACE INTO daily_price [OK]
[2024-01-11 17:41] #0001 동화약품 (000020) : 1000 rows > REPLACE INTO daily_price2 [OK]
